In [4]:
#Importing Pacakages

import pandas as pd #Data Processing
from sklearn.preprocessing import StandardScaler #Data Normalization


In [7]:
#Import the dataset

df = pd.read_csv('creditcard.csv')
df.drop('Time', axis=1, inplace=True)
print(df.head())

         V1        V2        V3        V4        V5        V6        V7  \
0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9       V10  ...       V21       V22       V23       V24  \
0  0.098698  0.363787  0.090794  ... -0.018307  0.277838 -0.110474  0.066928   
1  0.085102 -0.255425 -0.166974  ... -0.225775 -0.638672  0.101288 -0.339846   
2  0.247676 -1.514654  0.207643  ...  0.247998  0.771679  0.909412 -0.689281   
3  0.377436 -1.387024 -0.054952  ... -0.108300  0.005274 -0.190321 -1.175575   
4 -0.270533  0.817739  0.753074  ... -0.009431  0.798278 -0.137458  0.141267   

        V25       V26       V27       V28  Amount  Class  
0  0.1285

I have a look at the number of fraud and non-fraud cases in our dataset as well as  also calculate the percentage of fraud cases in the total number of recorded transactions. 



In [8]:

cases = len(df)
nonfraud_count = len(df[df.Class == 0])
fraud_count = len(df[df.Class == 1])
fraud_percentage = round(fraud_count/nonfraud_count*100,2)

print("Total Transactions is : ",cases)
print("Noramal Transactions is : ",nonfraud_count)
print("Abnormal Transactions is : ",fraud_count)
print("Total Fraud Percentage is : ",fraud_percentage)

Total Transactions is :  284807
Noramal Transactions is :  284315
Abnormal Transactions is :  492
Total Fraud Percentage is :  0.17


There are just 492 fraud incidents out of 284,807 samples, or 0.17 percent of all samples. As a result, I may conclude that the data I am working with is very unbalanced and must be treated with caution while modelling and assessing it. Next, I will use Python's 'describe' function to gain a statistical perspective of both fraud and non-fraud transaction amount data.

The following line of code can also be used to check for null values.

In [27]:
df.isnull().values.any()

False

In the credit card dataset, look for null values. Fortunately, our dataset contains no null or NaN values.

The "Amount" feature is the one I was most interested in. The following is a summary of the feature.

In [9]:
nonfraud_count = df[df.Class == 0]
fraud_count = df[df.Class == 1]

print("Nonfraud transaction statistics \n")
print(nonfraud_count.Amount.describe(),"\n")

print("fraud transaction statistics \n ")
print(fraud_count.Amount.describe())

Nonfraud transaction statistics 

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64 

fraud transaction statistics 
 
count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64


When looking at the data, it's clear that the 'Amount' variable's values vary greatly when compared to the other variables. We may normalise it in Python using the 'StandardScaler' function to lessen its large range of values.


In [18]:
sc = StandardScaler()
amount = df['Amount'].values

df['Amount'] = sc.fit_transform(amount.reshape(-1, 1))
print(df['Amount'].head(10))


0     0.244964
1    -0.342475
2     1.160686
3     0.140534
4    -0.073403
5    -0.338556
6    -0.333279
7    -0.190107
8     0.019392
9    -0.338516
10   -0.322044
11   -0.313289
12    0.132538
13   -0.243282
14   -0.118142
15   -0.289300
16   -0.301294
17   -0.349671
18   -0.166119
19   -0.333239
Name: Amount, dtype: float64
